In [2]:
import os
import sys
import torch
import numpy as np
from torch.distributions import MultivariateNormal
from scipy.stats import invwishart
from numpy.linalg import inv, det

from deep_fields.models.random_fields.blocks_utils import calculate_determinant_and_inverse_covariance_history
from deep_fields.models.random_fields.blocks_utils import obtain_location_index_to_realization

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [3]:
total_assets_in_history = 30
birth_numbers = [5,1,2,2,2,4,2,2,5,5]
birth_numbers = np.array(birth_numbers)
assets_in_the_market = birth_numbers.cumsum()
number_of_realizations = len(assets_in_the_market)
location_index_to_realization = obtain_location_index_to_realization(birth_numbers)

In [5]:
#generate covariance matrix prior
nu = total_assets_in_history + 1.
Psi = np.random.rand(total_assets_in_history, total_assets_in_history)
Psi = np.dot(Psi, Psi.transpose())

a_J = np.ones(total_assets_in_history)*10.
b_J = 1.

lambda_ = 1/b_J
mu_0 = a_J

IW = invwishart(nu,Psi)
covariance_matrix = IW.rvs()
mu = torch.Tensor(np.ones(total_assets_in_history)*0.01)

In [24]:
covariance_matrix.mean()

5.857191902099562

In [25]:
covariance_matrix.std()

2.933792932774975

In [26]:
covariance_matrix.min()

1.1156843384934176

In [27]:
covariance_matrix.max()

15.251514322061205

In [55]:
block_size = 5
blocks_extra_size = 2
block_full_size = block_size + blocks_extra_size

In [56]:
block_00 = covariance_matrix[:block_size,:block_size]
block_01 = covariance_matrix[:block_size,block_size:block_full_size]
block_10 = covariance_matrix[block_size:block_full_size,:block_size]
block_11 = covariance_matrix[block_size:block_full_size:,block_size:block_full_size]

In [57]:
block_0 = np.hstack([block_00,block_01])
block_1 = np.hstack([block_10,block_11])
complete_block = np.vstack([block_0,block_1])

In [58]:
A = block_00
B = block_01
C = block_10
D = block_11
A_inv = inv(A)

In [59]:
det(complete_block)

0.04606934352748349

In [60]:
blocks_multiplications = D - np.dot(C, np.dot(A_inv, B))

In [61]:
A.shape

(5, 5)

In [62]:
det(A)

0.39134824668432294

In [63]:
determinant = det(A) * det(blocks_multiplications)
determinant

0.04606934352748422